# **[HW24] NaiveBayes Classifier**
1. Requirements
2. Data Preprocessing
3. Model Training
4. Evaluation

## 1. Requirements

#### 1.1 필요한 패키지를 설치(install) 및 import 합니다.

In [ ]:
# 한국어 전처리 라이브러리 
!pip install konlpy

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
from tqdm import tqdm
from collections import defaultdict
import math

# POS(Part of Speech) tagger
from konlpy import tag  #품사태거

#### 1.2 Train data 와 test data 를 준비합니다.

In [ ]:
data = {}
# training data. input text 와 정답 label (긍정(1), 부정(0)) 로 구성.

data['train'] = [{'text': "정말 재미있습니다. 추천합니다."},
                {'text': "기대했던 것보단 별로였네요."},
                {'text': "지루해서 다시 보고 싶다는 생각이 안 드네요."},
                {'text': "완전 최고입니다 ! 다시 보고 싶습니다."},
                {'text': "연기도 연출도 다 만족스러웠습니다."},
                {'text': "연기가 좀 별로였습니다."},
                {'text': "연출도 좋았고 배우분들 연기도 최고입니다."},
                {'text': "기념일에 방문했는데 연기도 연출도 다 좋았습니다."},
                {'text': "전반적으로 지루했습니다. 저는 별로였네요."},
                {'text': "CG에 조금 더 신경 썼으면 좋겠습니다."}
                ]
# test data
data['test'] = [{'text': "최고입니다. 또 보고 싶네요."},
                {'text': "별로였습니다. 되도록 보지 마세요."},
                {'text': "다른 분들께 추천드릴 수 있을 만큼 연출도 연기도 만족했습니다."},
                {'text': "연기가 좀 더 개선되었으면 좋겠습니다."}
                ]

train_labels = [1, 0, 0, 1, 1, 0, 1, 1, 0, 0]
test_labels = [1, 0, 1, 0]

### 2. Data Preprocessing


#### 2.1 한글 형태소 분석기를 이용해서 주어진 데이터를 tokenize 합니다.

오픈소스 형태소 분석기를 제공하는 파이썬 패키지 KoNLPy에서 제공하는 [꼬꼬마(Kkma) 형태소 분석기](https://konlpy.org/en/v0.5.2/api/konlpy.tag/#module-konlpy.tag._kkma)를 이용하여 tokenize 합니다.

In [ ]:
# 형태소 분석기 선언
morph_analyzer = tag.Kkma() 

In [ ]:
# tokenization 함수 정의
def tokenization(data, morph_analyzer):
    '''
    (input) data: list of data examples.
            morph_analyzer: morphological analyzer.
    (output) tokenized_data: list of tokenized data examples.
    '''
    tokenized_data = []

    for example in tqdm(data):
        tokens = morph_analyzer.morphs(example['text']) #str 단위 받아서 morphs 메소드 실행. list안에 str형태로 형태소 분석.
        tokenized_data.append(tokens) #tokenized_data는 이차원 리스트로 data의 문장의 갯수의 행을 가지고 있음.

    return tokenized_data

In [ ]:
# tokenization 함수를 이용한 데이터 tokenization
tokenized_data = {}

tokenized_data['train'] = tokenization(data['train'], morph_analyzer)
tokenized_data['test'] = tokenization(data['test'], morph_analyzer)

100%|██████████| 4/4 [00:00<00:00, 38.51it/s]


In [ ]:
# tokenized_data 확인
tokenized_data['train']

[['정말', '재미있', '습니다', '.', '추천', '하', 'ㅂ니다', '.'],
 ['기대', '하', '었', '더', 'ㄴ', '것', '보다', 'ㄴ', '별', '로', '이', '었', '네요', '.'],
 ['지루', '하', '어서', '다시', '보', '고', '싶', '다는', '생각', '이', '안', '들', '네요', '.'],
 ['완전', '최고', '이', 'ㅂ니다', '!', '다시', '보', '고', '싶', '습니다', '.'],
 ['연기', '도', '연출', '도', '다', '만족', '스럽', '었', '습니다', '.'],
 ['연기', '가', '좀', '별', '로', '이', '었', '습니다', '.'],
 ['연출', '도', '좋', '았', '고', '배우', '분', '들', '연기', '도', '최고', '이', 'ㅂ니다', '.'],
 ['기념일',
  '에',
  '방문',
  '하',
  '었',
  '는데',
  '연기',
  '도',
  '연출',
  '도',
  '다',
  '좋',
  '았',
  '습니다',
  '.'],
 ['전반적',
  '으로',
  '지루',
  '하',
  '었',
  '습니다',
  '.',
  '저',
  '는',
  '별',
  '로',
  '이',
  '었',
  '네요',
  '.'],
 ['CG', '에', '조금', '더', '신경', '쓰', '었', '으면', '좋', '겠', '습니다', '.']]

#### 2.2 tokenization 결과를 이용해서 word to index dictionary 를 생성합니다.


In [ ]:
# train data의 tokenization 결과에서 unique token만 남긴 set으로 변환
tokens = [token for i in range(len(tokenized_data['train'])) for token in tokenized_data['train'][i] ] #형태소 분리 완료한 train data의 문장 하나씩 꺼내서 1차원 리스트에 concat
print(tokens)

unique_train_tokens = set(tokens) #1차원 리스트에 train data의 유일한 형태소만 남기고 set형태로 변환.

# NaiveBayes Classifier의 input에 들어갈 word의 index를 반환해주는 dictionary를 생성
word2index = defaultdict() # key: word, value: index of word
idx = 0
for token in tqdm(unique_train_tokens): #train data 유일토큰을 담는 set에서 유일토큰을 하나씩 꺼내서 인덱스를 매겨준다.
    word2index[token] = idx #형태소마다 각자 인덱스를 지니게 된다. 인덱스는 0부터 시작함.
    idx += 1

['정말', '재미있', '습니다', '.', '추천', '하', 'ㅂ니다', '.', '기대', '하', '었', '더', 'ㄴ', '것', '보다', 'ㄴ', '별', '로', '이', '었', '네요', '.', '지루', '하', '어서', '다시', '보', '고', '싶', '다는', '생각', '이', '안', '들', '네요', '.', '완전', '최고', '이', 'ㅂ니다', '!', '다시', '보', '고', '싶', '습니다', '.', '연기', '도', '연출', '도', '다', '만족', '스럽', '었', '습니다', '.', '연기', '가', '좀', '별', '로', '이', '었', '습니다', '.', '연출', '도', '좋', '았', '고', '배우', '분', '들', '연기', '도', '최고', '이', 'ㅂ니다', '.', '기념일', '에', '방문', '하', '었', '는데', '연기', '도', '연출', '도', '다', '좋', '았', '습니다', '.', '전반적', '으로', '지루', '하', '었', '습니다', '.', '저', '는', '별', '로', '이', '었', '네요', '.', 'CG', '에', '조금', '더', '신경', '쓰', '었', '으면', '좋', '겠', '습니다', '.']


100%|██████████| 56/56 [00:00<00:00, 524288.00it/s]


### 3. Model Training

#### 3.1 NaiveBayes Classifier 모델 클래스를 구현합니다.


In [ ]:
class NaiveBayesClassifier():
    def __init__(self, word2index, k=0.1):
        """
        (input) word2index: mapping a word to a pre-assigned index.
        """
        self.k = k # for smoothing
        self.word2index = word2index
        self.priors = {} # Prior probability for each class, P(c) //// class : 0 or 1
        self.likelihoods = {} # Likelihood for each token, P(d|c)

    def _set_priors(self, labels):
        """
        Set prior probability for each class, P(c).
        Count the number of each class and calculate P(c) for each class.
        """
        
        class_counts = defaultdict(int)
        ############################ ANSWER HERE ################################
        # TODO 1: Count the number of each class
        for label in tqdm(labels): #prior은 good bad 클래스의 확률이므로 라벨의 갯수로 구할 수 있다.
            class_counts[label] += 1
        # TODO 2: For each class, calcuate P(c)
        for label,count in class_counts.items():
            self.priors[label] = class_counts[label] / len(labels) #해당 클래스에 속하는 문장의 갯수에서 전체 라벨의 갯수(문장의 갯수) 를 나누면 prior를 구할 수 있다.
        #########################################################################        

    def _set_likelihoods(self, tokens, labels):
        """
        Set likelihood for each token, P(d|c).
        First, count the number of each class for each token.
        Then, calculate P(d|c) for a given class and token.
        """
        token_dists = {}
        class_counts = defaultdict(int)

        for i, label in enumerate(tqdm(labels)):#train하는 도큐먼트 수만큼 만복(문장의 수만큼 반복.)
            count = 0

            ############################ ANSWER HERE ################################
            # TODO: Count the number of each class for each token.
            # tokens : tokenized_data['train'] -> 2d array.   [[token,token,...,token] , [token,...,token] , ... [token,...,token]]
            # word2index :      word2index     -> dictionary. {token1 : 2 , token2 : 3, ....} frequency per unique token
            # token_dists:      token_dists 
            for token in tokens[i]: #한 문장 안에서 토큰을 하나 뽑는다.
                if token in self.word2index: # 그 토큰이 인덱스로 치환 가능한 토큰이면서(?)
                    if token not in token_dists: #유일한게 뽑은 값일때. initialize
                        token_dists[token] = {0:0,1:0} #initialize
                    token_dists[token][label] += 1 # 각 토큰이 등장할때 마다. 그 토큰이 속해있는 문장의 클래스가 0일때 혹은 1일때 각각 갯수를 구한다.
            #########################################################################        

                count += 1 #토큰의 갯수?
            #count+=1 #얘로 하면 문장의 갯수를 센다.
            class_counts[label] += count #클래스별로 토큰의 총 개수.

        for token, dist in tqdm(token_dists.items()): #토큰 하나씩 꺼낸다.
            if token not in self.likelihoods: 
                self.likelihoods[token] = {
                    0: (token_dists[token][0]+ self.k) / (class_counts[0] + len(self.word2index)* self.k), # 클래스별로 특정클래스에 속하는 모든 문장의 토큰의 총개수 분에 해당 클래스인 모든 문장에서의 특정 토큰의 갯수를 구한다.
                    1: (token_dists[token][1]+ self.k) / (class_counts[1] + len(self.word2index)* self.k),
                }

    def train(self, input_tokens, labels):
        """
        (input) input_tokens: list of tokenized train data.
                labels: train labels for each sentence/document.
        """
        self._set_priors(labels)
        self._set_likelihoods(input_tokens, labels)

    def inference(self, input_tokens):
        """
        (input) input_tokens: list_of tokenized test data.
        """
        log_prob_0 = 0.0
        log_prob_1 = 0.0

        for token in input_tokens: #kkma 토큰화 완료된 1d array에서 토큰을 하나씩 꺼낸다.
            if token in self.likelihoods: #우리가 일전에 학습했던 토큰 pool에 속한 토큰이면면
                log_prob_0 += math.log(self.likelihoods[token][0]) # 0일확률
                log_prob_1 += math.log(self.likelihoods[token][1])# 1일확률 구한다.
#문맥은 고려 안하나 보네... 그냥 형태소의 성격만 판단한다.
        log_prob_0 += math.log(self.priors[0]) #
        log_prob_1 += math.log(self.priors[1])

        if log_prob_0 >= log_prob_1:###
            return 0
        else:
            return 1

#### 3.2 주어진 학습 데이터에 대해 문장 분류 모델을 학습시킵니다.

In [ ]:
# 문장 분류 모델 선언 및 학습
classifier = NaiveBayesClassifier(word2index)
classifier.train(tokenized_data['train'], train_labels)

100%|██████████| 56/56 [00:00<00:00, 360246.97it/s]


In [ ]:
classifier.likelihoods

{'정말': {0: 0.0014367816091954025, 1: 0.01729559748427673},
 '재미있': {0: 0.0014367816091954025, 1: 0.01729559748427673},
 '습니다': {0: 0.04454022988505748, 1: 0.06446540880503145},
 '.': {0: 0.08764367816091954, 1: 0.09591194968553458},
 '추천': {0: 0.0014367816091954025, 1: 0.01729559748427673},
 '하': {0: 0.04454022988505748, 1: 0.0330188679245283},
 'ㅂ니다': {0: 0.0014367816091954025, 1: 0.04874213836477988},
 '기대': {0: 0.015804597701149427, 1: 0.0015723270440251573},
 '었': {0: 0.08764367816091954, 1: 0.0330188679245283},
 '더': {0: 0.030172413793103453, 1: 0.0015723270440251573},
 'ㄴ': {0: 0.030172413793103453, 1: 0.0015723270440251573},
 '것': {0: 0.015804597701149427, 1: 0.0015723270440251573},
 '보다': {0: 0.015804597701149427, 1: 0.0015723270440251573},
 '별': {0: 0.04454022988505748, 1: 0.0015723270440251573},
 '로': {0: 0.04454022988505748, 1: 0.0015723270440251573},
 '이': {0: 0.05890804597701149, 1: 0.0330188679245283},
 '네요': {0: 0.04454022988505748, 1: 0.0015723270440251573},
 '지루': {0: 

### 4. Evaluation

각각의 Test 데이터에 대해 정답값을 추론하고 Accuracy를 구합니다.

In [ ]:
# Test 데이터 inference
preds = []
for test_tokens in tqdm(tokenized_data['test']):
    pred = classifier.inference(test_tokens)
    preds.append(pred)

100%|██████████| 4/4 [00:00<00:00, 2704.26it/s]


In [ ]:
# Accuracy 측정
from sklearn.metrics import accuracy_score

print(accuracy_score(test_labels, preds))

1.0
